# Yoonhyuck WOO / Purdue University_Computer and Information Technology
# Assignment 2: Exploring fairness during model training
# Professor: Dr. Pradhan

## Date: 2.2 - 5:00pm 2/16/2024 (EST)

We discussed a number of fairness metrics in the context of binary classification. Building up on the previous assignment, we will explore aspects of fairness in model predictions while training our model.

In this lab, **we will detect bias that may be introduced while training classifiers.**

This notebook has four stages in which we will: 
1. Import the data
2. Implement a few pre-processing steps, and inspect the data (compute base rates on original data)
3. Train a classifier to predict credit using the original data, and measure bias using fairness metrics including statistical parity and equalized odds. 
4. Train a classifier to predict credit using the original data without sensitive features, and measure bias using fairness metrics including statistical parity and equalized odds. 
5. (Extra credit) Use IBM's AIF360 toolkit to compute fairness metrics.

In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline

import random
random.seed(6)

import sys
import warnings

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

### Step 1: Load data

For this assignment, we will work with the German Credit dataset which has been provided with this notebook (the dataset can also be downloaded from the UCI ML repository). The dataset has demographic and financial information for about 1,000 individuals, and the task for this dataset is to predict whether an individual is a good credit risk or a bad credit risk. More information about the dataset can be found here: https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data.

Load the dataset and check the first few rows:

In [2]:
cols = ['status', 'duration', 'credit_hist', 'purpose', 'credit_amt', 'savings', 'employment',\
            'install_rate', 'personal_status', 'debtors', 'residence', 'property', 'age', 'install_plans',\
            'housing', 'num_credits', 'job', 'num_liable', 'telephone', 'foreign_worker', 'credit']
data_df = pd.read_table('D:\DOWNLOAD\statlog+german+credit+data/german.data', names=cols, sep=" ", index_col=False)
y = data_df['credit']

print("Shape: ", data_df.shape)
data_df.head(5)

Shape:  (1000, 21)


,status,duration,credit_hist,purpose,credit_amt,savings,employment,install_rate,personal_status,debtors,...,property,age,install_plans,housing,num_credits,job,num_liable,telephone,foreign_worker,credit
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


### Step 2: Data preprocessing and data analysis

#### 2.1. Adding two new columns
Before training a model, repeat the data processing steps from assignment 1 here to create the "sex" and "age" columns.
<ol>
    <li> Write code to create the "sex" column and append to the dataframe. Consider sex=`male' as privileged. </li> 
    <li> Write code to replace the "age" column with discrete values. Consider age ≥ 25 as privileged. </li>
</ol>

As in assignment 1, we will train a machine learning model to predict good/bad credit. You are free to use any model but you may use the data preprocessing steps from assignment 1 to transform the data.

In [3]:
# Write code for data preparation here

# write code to create the "sex" column and append to the dataframe
'''
A91 : male   : divorced/separated
A93 : male   : single
A94 : male   : married/widowed

A92 : female : divorced/separated/married
A95 : female : single
'''

sex = data_df['personal_status']
sex_list = data_df['personal_status'].tolist() 

new_sex_list = []
for i in range(len(sex_list)):
    if sex_list[i] in ("A91", "A93", "A94"): 
        i = 'male'
        new_sex_list.append(i)
    else:
        i = 'female'
        new_sex_list.append(i)

        
data_df.insert(9, "sex", new_sex_list)

# write code to replace the "age" column with discrete values
age_threshold = 25

data_df["age"] = np.where(data_df["age"] >= age_threshold, "old", "young")

# print the first few rows
data_df.head(5)

,status,duration,credit_hist,purpose,credit_amt,savings,employment,install_rate,personal_status,sex,...,property,age,install_plans,housing,num_credits,job,num_liable,telephone,foreign_worker,credit
0,A11,6,A34,A43,1169,A65,A75,4,A93,male,...,A121,old,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,female,...,A121,young,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,male,...,A121,old,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,male,...,A122,old,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,male,...,A124,old,A143,A153,2,A173,2,A191,A201,2


#### 2.2. Sensitive attributes

In the class, we learnt about sensitive attributes for computing model fairness. In this assignment, we will compute different fairness metrics in two scenarios:

<ol>
    <li> considering only "age" as the sensitive attribute </li>
    <li> considering only "sex" as the sensitive attribute </li>
</ol>

In [4]:
pd.crosstab(data_df['sex'], data_df['age'], margins = True)

age,old,young,All
sex,,,
female,226,84,310
male,625,65,690
All,851,149,1000


In [5]:
pd.crosstab(data_df['credit'], data_df['age'], margins = True)

age,old,young,All
credit,,,
1,612,88,700
2,239,61,300
All,851,149,1000


In [6]:
pd.crosstab(data_df['credit'], data_df['sex'], margins = True)

sex,female,male,All
credit,,,
1,201,499,700
2,109,191,300
All,310,690,1000


In [7]:
crosstab = pd.crosstab(data_df['sex'], data_df['age'], margins = True)
print(crosstab["young"]["All"])

149


In [8]:
print("male_Good_Credit (previleged group):", len(data_df[(data_df['sex']=='male') & (data_df['credit']==1)]))
print("old_Good_Credit (previleged group):", len(data_df[(data_df['age']=='old') & (data_df['credit']==1)]))

print("female_Good_Credit (unprevileged group):", len(data_df[(data_df['sex']=='female') & (data_df['credit']==1)]))
print("young_Good_Credit (unprevileged group):", len(data_df[(data_df['age']=='young') & (data_df['credit']==1)]))

male_Good_Credit (previleged group): 499
old_Good_Credit (previleged group): 612
female_Good_Credit (unprevileged group): 201
young_Good_Credit (unprevileged group): 88


#### 2.3. Compute base rates on original data
Before we consider model fairness, let's first compare the base rates for the privileged and unprivileged groups with that of the entire data. Base rate indicates the fraction of data (or a subset of the data) that has positive outcomes. Base rates indicate class imbalance in the dataset.

In the following code block, compute the base rates for the entire dataset, and for the privileged and unprivileged groups.

# Note 
(Reference: https://towardsdatascience.com/a-tutorial-on-fairness-in-machine-learning-3ff8ba1040cb)

- X ∈ Rᵈ: quantified features of the applicant(e.g. education, work experience, college GPA, etc.).
- A ∈ {0, 1}: a binary sensitive attribute
- C :=c(X,A) ∈ {0,1}: binary predictor (e.g. good credit/bad credit), which makes decision based on a score R:=r(x, a) ∈ [0,1].
- Y ∈ {0, 1}: target variable(e.g. if the candidate is truly capable of the position).

We assume X, A, Y are generated from an underlying distribution D i.e. (X, A, Y) ~ D.

We also denote P₀ [c] := P [c | A= 0].

In [9]:
sex_previleged_group_positive = len(data_df[(data_df['sex']=='male') & (data_df['credit']==1)])
sex_unprevileged_group_positive = len(data_df[(data_df['sex']=='female') & (data_df['credit']==1)])

age_previleged_group_positive = len(data_df[(data_df['age']=='old') & (data_df['credit']==1)])
age_unprevileged_group_positive = len(data_df[(data_df['age']=='young') & (data_df['credit']==1)])

male_All = crosstab["All"]["male"]
female_All = crosstab["All"]["female"]
old_All = crosstab["old"]["All"]
young_All = crosstab["young"]["All"]

In [10]:
# Write code to compute base rates with respect to "age" as the sensitive attribute
sex_previleged_group_baserates = sex_previleged_group_positive / male_All
sex_unprevileged_group_baserates = sex_unprevileged_group_positive / female_All

print("sex_previleged_group_baserates", round(sex_previleged_group_baserates, 3))
print("sex_unprevileged_group_baserates", round(sex_unprevileged_group_baserates,3))

sex_previleged_group_baserates 0.723
sex_unprevileged_group_baserates 0.648


In [11]:
# Write code to compute base rates with respect to "sex" as the sensitive attribute
age_previleged_group_baserates = age_previleged_group_positive / old_All
age_unprevileged_group_baserates = age_unprevileged_group_positive / young_All

print("age_previleged_group_baserates", round(age_previleged_group_baserates, 3))
print("age_unprevileged_group_baserates", round(age_unprevileged_group_baserates,3))

age_previleged_group_baserates 0.719
age_unprevileged_group_baserates 0.591


**TODO:** Write in words if the value computed above (sex and age attribute) indicates favoring the privileged group or the unprivileged group.

|      | Sex | Age   |
| :---        |    :----:   |          ---: |
| **Privileged Group**  | 0.723       | 0.719   |
| **Unprevileged Group**   | 0.648       | 0.591      |

Based on the above table, I think the value favors privileged groups, both sex and age attributes; moreover, as we can see from the above table, both the privileged group and privileged values focusing on "sex" as the sensitive attribute show higher than the other.

### Step 3: Model training
Let's now prepare the setup for model training and evaluation

#### 3.1. Identify the training features and target variable

We will evaluate the learned model on the training data. Therefore, we will not split the data into train and test sets. We are also not concerned with optimizing the learned model for performance and, therefore, we will not need to create a validation dataset.

#### 3.2. Model training

In this part, we will set up our machine learning model and fit the model using the training data. Below, we have learnt a logistic regression model, but **you are free to choose any model**. Since we are not concerned with optimal model performance here, there is no need to perform hyperparameter tuning.

In [12]:
# write code to preprocess the data
data_df['age'].replace('old', 1, inplace=True)
data_df['age'].replace('young', 0, inplace=True)

# iterating the columns
col_names = []
datatype = []
for col in data_df.columns:
    col_names.append(col)
    
for dtype in data_df.dtypes:
    datatype.append(dtype)
    
def convert(col_names, datatype):
    res_dict = {}
    for i in range(len(col_names)):
        res_dict[col_names[i]] = datatype[i]
        i = i+1
        
    return res_dict

dictionary = convert(col_names, datatype)

#import label encoder
from sklearn import preprocessing 
#make an instance of Label Encoder
label_encoder = preprocessing.LabelEncoder()

for cols in col_names:

    if dictionary[cols] == 'object':
        data_df[cols] = label_encoder.fit_transform(data_df[cols])
    else:
        pass


# print the first few rows of the data
data_df.head()                                                          

,status,duration,credit_hist,purpose,credit_amt,savings,employment,install_rate,personal_status,sex,...,property,age,install_plans,housing,num_credits,job,num_liable,telephone,foreign_worker,credit
0,0,6,4,4,1169,4,4,4,2,1,...,0,1,2,1,2,2,1,1,0,1
1,1,48,2,4,5951,0,2,2,1,0,...,0,0,2,1,1,2,1,0,0,2
2,3,12,4,7,2096,0,3,2,2,1,...,0,1,2,1,1,1,2,0,0,1
3,0,42,2,3,7882,0,3,2,2,1,...,1,1,2,2,1,2,2,0,0,1
4,0,24,3,0,4870,0,2,3,2,1,...,3,1,2,2,2,2,2,0,0,2


In [13]:
# Set up the data for training
x_train = data_df.drop("credit", axis=1)
y_train = data_df.credit.replace({2:0}) #1 = Good, 2= Bad credit risk
print("Training Outcomes: \n", y_train.value_counts())

Training Outcomes: 
 credit
1    700
0    300
Name: count, dtype: int64


In [14]:
model = LogisticRegression(C=1000, penalty="l1", solver='liblinear')
    
# Fit the model using the training data
model = model.fit(x_train, y_train, sample_weight=None)

# Observe the class labels
print(model.classes_)

# Calculate predicted values on training data
y_pred = model.predict(x_train)

# You can also compute the probabilties for the two clases using the predict_proba function
y_pred_probs = model.predict_proba(x_train)

[0 1]


#### 3.3. Model evaluation

Compute the model accuracy using sklearn's `accuracy_score` function

In [15]:
accuracy = accuracy_score(y_train, y_pred)
print("accuracy_score", accuracy)

accuracy_score 0.771


#### 3.4. Evaluate model fairness 
In the class, we went over a number of fairness metrics. In this part of the assignment, you will write functions to compute statistical parity and equalized odds. Recall that both of these metrics use the model predictions $\hat{Y}$ (computed as $y\_pred$ above).

Let $S=0$ indicate the unprivileged group and $S=1$ indicate the privileged group. FPR and FNR represent the false positive rate and false negative rate respectively. Then, we can compute:

**Statistical parity** = $P(\hat{Y} = 1 | S = 0) - P(\hat{Y} = 1 | S = 1)$ 

### Statistical parity

In [16]:
data_df.insert(22, "y_pred", y_pred)
data_df.head(5)

,status,duration,credit_hist,purpose,credit_amt,savings,employment,install_rate,personal_status,sex,...,age,install_plans,housing,num_credits,job,num_liable,telephone,foreign_worker,credit,y_pred
0,0,6,4,4,1169,4,4,4,2,1,...,1,2,1,2,2,1,1,0,1,1
1,1,48,2,4,5951,0,2,2,1,0,...,0,2,1,1,2,1,0,0,2,0
2,3,12,4,7,2096,0,3,2,2,1,...,1,2,1,1,1,2,0,0,1,1
3,0,42,2,3,7882,0,3,2,2,1,...,1,2,2,1,2,2,0,0,1,1
4,0,24,3,0,4870,0,2,3,2,1,...,1,2,2,2,2,2,0,0,2,0


In [17]:
sex_pred_p = len(data_df[(data_df['sex']==1) & (data_df['y_pred']==1)])
print(sex_pred_p)
sex_pred_unp = len(data_df[(data_df['sex']==0) & (data_df['y_pred']==1)])
print(sex_pred_unp)

age_pred_p = len(data_df[(data_df['age']==1) & (data_df['y_pred']==1)])
print(age_pred_p)
age_pred_unp = len(data_df[(data_df['age']==0) & (data_df['y_pred']==1)])
print(age_pred_unp)

563
214
685
92


In [18]:
# Report statistical parity and equalized odds using "sex" as the sensitive attribute
sex_statistical_parity = (sex_pred_unp / female_All) - (sex_pred_p / male_All)

# Report statistical parity and equalized odds using "age" as the sensitive attribute
age_statistical_parity = (age_pred_unp / young_All) - (age_pred_p / old_All)

In [19]:
print('sex_statistical_parity', round(sex_statistical_parity, 7))
print('age_statistical_parity', round(age_statistical_parity, 7))

sex_statistical_parity -0.1256194
age_statistical_parity -0.1874857


**Equalized odds** $\dfrac{(FPR_{S=0} - FPR_{S=1}) + (FNR_{S=0} - FNR_{S=1})}{2}$

where $FPR = P(\hat{Y}=1 | Y=0)$ and $FNR = P(\hat{Y}=0 | Y=1)$

### Equalized Odds

In [20]:
sex_prev_fact = data_df[(data_df['sex']==1)]
sex_prev_fact = sex_prev_fact.credit.replace({2:0})

sex_prev_pred = data_df[(data_df['sex']==1)]
sex_prev_pred = sex_prev_pred.y_pred

sex_unprev_fact = data_df[(data_df['sex']==0)]
sex_unprev_fact = sex_unprev_fact.credit.replace({2:0})

sex_unprev_pred = data_df[(data_df['sex']==0)]
sex_unprev_pred = sex_unprev_pred.y_pred

# ==============================================================

age_prev_fact = data_df[(data_df['age']==1)]
age_prev_fact = age_prev_fact.credit.replace({2:0})

age_prev_pred = data_df[(data_df['age']==1)]
age_prev_pred = age_prev_pred.y_pred

age_unprev_fact = data_df[(data_df['age']==0)]
age_unprev_fact = age_unprev_fact.credit.replace({2:0})

age_unprev_pred = data_df[(data_df['age']==0)]
age_unprev_pred = age_unprev_pred.y_pred


In [21]:
sex_prev_conf_matrix = confusion_matrix(sex_prev_fact, sex_prev_pred, labels=[0, 1])
sex_unprev_conf_matrix = confusion_matrix(sex_unprev_fact, sex_unprev_pred, labels=[0, 1])

age_prev_conf_matrix = confusion_matrix(age_prev_fact, age_prev_pred, labels=[0, 1])
age_unprev_conf_matrix = confusion_matrix(age_unprev_fact, age_unprev_pred, labels=[0, 1])

In [22]:
print(sex_prev_conf_matrix)
print(" ")
print(sex_unprev_conf_matrix)
print(" ")
print(age_prev_conf_matrix)
print(" ")
print(age_unprev_conf_matrix)


[[ 80 111]
 [ 47 452]]
 
[[ 67  42]
 [ 29 172]]
 
[[109 130]
 [ 57 555]]
 
[[38 23]
 [19 69]]


In [23]:
disp = ConfusionMatrixDisplay(confusion_matrix=sex_prev_conf_matrix, display_labels=["bad_credit (0)","good_credit (1)"])
disp2 = ConfusionMatrixDisplay(confusion_matrix=sex_unprev_conf_matrix, display_labels=["bad_credit (0)","good_credit (1)"])
disp3 = ConfusionMatrixDisplay(confusion_matrix=age_prev_conf_matrix, display_labels=["bad_credit (0)","good_credit (1)"])
disp4 = ConfusionMatrixDisplay(confusion_matrix=age_unprev_conf_matrix, display_labels=["bad_credit (0)","good_credit (1)"])

In [24]:
sex_prev_TN = sex_prev_conf_matrix[0][0]
sex_prev_FN = sex_prev_conf_matrix[1][0]
sex_prev_TP = sex_prev_conf_matrix[1][1]
sex_prev_FP = sex_prev_conf_matrix[0][1]

sex_unprev_TN = sex_unprev_conf_matrix[0][0]
sex_unprev_FN = sex_unprev_conf_matrix[1][0]
sex_unprev_TP = sex_unprev_conf_matrix[1][1]
sex_unprev_FP = sex_unprev_conf_matrix[0][1]

age_prev_TN = age_prev_conf_matrix[0][0]
age_prev_FN = age_prev_conf_matrix[1][0]
age_prev_TP = age_prev_conf_matrix[1][1]
age_prev_FP = age_prev_conf_matrix[0][1]

age_unprev_TN = age_unprev_conf_matrix[0][0]
age_unprev_FN = age_unprev_conf_matrix[1][0]
age_unprev_TP = age_unprev_conf_matrix[1][1]
age_unprev_FP = age_unprev_conf_matrix[0][1]

In [25]:
sex_prev_FPR = sex_prev_FP / (sex_prev_FP + sex_prev_TN)
sex_prev_FNR = sex_prev_FN / (sex_prev_FN + sex_prev_TP)

sex_unprev_FPR = sex_unprev_FP / (sex_unprev_FP + sex_unprev_TN)
sex_unprev_FNR = sex_unprev_FN / (sex_unprev_FN + sex_unprev_TP)

age_prev_FPR = age_prev_FP / (age_prev_FP + age_prev_TN)
age_prev_FNR = age_prev_FN / (age_prev_FN + age_prev_TP)

age_unprev_FPR = age_unprev_FP / (age_unprev_FP + age_unprev_TN)
age_unprev_FNR = age_unprev_FN / (age_unprev_FN + age_unprev_TP)

In [26]:
import fairlearn
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference, equalized_odds_ratio
print('demographic_parity_sex', demographic_parity_difference(data_df['credit'],
                                    y_pred,
                                    sensitive_features=data_df['sex']))

print('demographic_parity_age', demographic_parity_difference(data_df['credit'],
                                    y_pred,
                                    sensitive_features=data_df['age']))

print(equalized_odds_difference(y_train,
                                    y_pred,
                                    sensitive_features=data_df['sex']))
print(equalized_odds_difference(y_train,
                                    y_pred,
                                    sensitive_features=data_df['age']))

demographic_parity_sex 0.125619448340346
demographic_parity_age 0.1874857057232313
0.19583073154330177
0.1668838740654366


In [27]:
sex_prev_FPR = sex_prev_FP / (sex_prev_FP + sex_prev_TN)
sex_prev_FNR = sex_prev_FN / (sex_prev_FN + sex_prev_TP)

sex_unprev_FPR = sex_unprev_FP / (sex_unprev_FP + sex_unprev_TN)
sex_unprev_FNR = sex_unprev_FN / (sex_unprev_FN + sex_unprev_TP)

age_prev_FPR = age_prev_FP / (age_prev_FP + age_prev_TN)
age_prev_FNR = age_prev_FN / (age_prev_FN + age_prev_TP)

age_unprev_FPR = age_unprev_FP / (age_unprev_FP + age_unprev_TN)
age_unprev_FNR = age_unprev_FN / (age_unprev_FN + age_unprev_TP)

In [28]:
# Write function to compute statistical parity over model predictions with given sensitive attribute
# statistical_parity = 
# Write function to compute equalized odds with given sensitive attribute
# equalized_odds = 

# sex_equalized_odds = ((sex_unprev_FPR-sex_prev_FPR)+(sex_unprev_FNR-sex_prev_FNR))
sex_equalized_odds = ((sex_unprev_FPR-sex_prev_FPR)+(sex_unprev_FNR-sex_prev_FNR)) / 2


age_equalized_odds = ((age_unprev_FPR-age_prev_FPR)+(age_unprev_FNR-age_prev_FNR)) / 2

print('sex_equalized_odds', round(sex_equalized_odds, 7))
print('age_equalized_odds', round(age_equalized_odds, 7))

sex_equalized_odds -0.0728703
age_equalized_odds -0.022056


**TODO:** What can you say about the model being more or less biased than the original credit scores?

|      | Sex | Age   |
| :---        |    :----:   |          ---: |
| **statistical_parity**  | -0.1256194       | -0.1874857   |
| **equalized_odds**   | -0.0728703       | -0.022056      |

The ideal value of both the statistical parity and equalized odds is 0. However, both of them don't so that we can expect unfairness.

First of all, in the aspect of statistical parity, both attributes show negative values, which means the unprivileged group has more disadvantages, and we can expect that the model is more biased than the original credit scores.

Second, the aspect of the equalized odds, As we can see above table, it also shows negative values, which means a higher benefit for the privileged group. Moreover, according to the AIf360 page, "The equalized odds means that difference of true positive rates between the unprivileged and the privileged groups," so we can expect that the model is more biased.

### Step 4: Model training without using the sensitive feature

We discussed in the class how removing a sensitive attribute is not enough to generate models that result in fair predictions. In this step, we will see if that is indeed true in action.

4.1. In the following, we will build models trained without the sensitive attribute (once for the "sex" attribute and one for the "age" attribute). 

In [29]:
# Drop the sensitive attribute from training data
temp_x_train_age = x_train.copy()
temp_x_train_sex = x_train.copy()
x_train_no_age = temp_x_train_age.drop('age', axis=1)
x_train_no_sex = temp_x_train_sex.drop('sex', axis=1)

model1 = LogisticRegression(C=1000, penalty="l1", solver='liblinear')

model2 = LogisticRegression(C=1000, penalty="l1", solver='liblinear')

# Fit the model using the training data
model_no_age = model1.fit(x_train_no_age, y_train, sample_weight=None)
model_no_sex = model2.fit(x_train_no_sex, y_train, sample_weight=None)

# Observe the class labels
print(model.classes_)

# Calculate the predicted values and probabilities on the updated training data
y_pred_no_age = model_no_age.predict(x_train_no_age)
y_pred_no_sex = model_no_sex.predict(x_train_no_sex)

[0 1]


In [30]:
print("y_pred_no_age_score", accuracy_score(y_train, y_pred_no_age))
print("y_pred_no_sex_score", accuracy_score(y_train, y_pred_no_sex))

y_pred_no_age_score 0.774
y_pred_no_sex_score 0.77


In [31]:
data_df.insert(23, "y_pred_no_age", y_pred_no_age)
data_df.insert(24, "y_pred_no_sex", y_pred_no_sex)

data_df.head(5)

,status,duration,credit_hist,purpose,credit_amt,savings,employment,install_rate,personal_status,sex,...,housing,num_credits,job,num_liable,telephone,foreign_worker,credit,y_pred,y_pred_no_age,y_pred_no_sex
0,0,6,4,4,1169,4,4,4,2,1,...,1,2,2,1,1,0,1,1,1,1
1,1,48,2,4,5951,0,2,2,1,0,...,1,1,2,1,0,0,2,0,0,0
2,3,12,4,7,2096,0,3,2,2,1,...,1,1,1,2,0,0,1,1,1,1
3,0,42,2,3,7882,0,3,2,2,1,...,2,1,2,2,0,0,1,1,1,1
4,0,24,3,0,4870,0,2,3,2,1,...,2,2,2,2,0,0,2,0,0,0


In [32]:
sex_pred_no_age_p = len(data_df[(data_df['sex']==1) & (data_df['y_pred_no_age']==1)])
print(sex_pred_no_age_p)
sex_pred_no_age_unp = len(data_df[(data_df['sex']==0) & (data_df['y_pred_no_age']==1)])
print(sex_pred_no_age_unp)

sex_pred_no_sex_p = len(data_df[(data_df['sex']==1) & (data_df['y_pred_no_sex']==1)])
print(sex_pred_no_sex_p)
sex_pred_no_sex_unp = len(data_df[(data_df['sex']==0) & (data_df['y_pred_no_sex']==1)])
print(sex_pred_no_sex_unp)

age_pred_no_age_p = len(data_df[(data_df['age']==1) & (data_df['y_pred_no_age']==1)])
print(age_pred_no_age_p)
age_pred_no_age_unp = len(data_df[(data_df['age']==0) & (data_df['y_pred_no_age']==1)])
print(age_pred_no_age_unp)

age_pred_no_sex_p = len(data_df[(data_df['age']==1) & (data_df['y_pred_no_sex']==1)])
print(age_pred_no_sex_p)
age_pred_no_sex_unp = len(data_df[(data_df['age']==0) & (data_df['y_pred_no_sex']==1)])
print(age_pred_no_sex_unp)

562
216
558
218
679
99
684
92


#### 4.2. Evaluating updated model

Now, let's evaluate the predictions obtained from the updated model using the functions that your wrote in Step 3.4. Although the sensitive attribute has not been used in training the model, the group memberships will be used to compute the fairness metrics.

In [52]:
sex_prev_fact = data_df[(data_df['sex']==1)]
sex_prev_fact = sex_prev_fact.credit.replace({2:0})

sex_prev_pred_no_age = data_df[(data_df['sex']==1)]
sex_prev_pred_no_age = sex_prev_pred_no_age.y_pred_no_age

sex_prev_pred_no_sex = data_df[(data_df['sex']==1)]
sex_prev_pred_no_sex = sex_prev_pred_no_sex.y_pred_no_sex
# --------------------------------------------------------------------------
sex_unprev_fact = data_df[(data_df['sex']==0)]
sex_unprev_fact = sex_unprev_fact.credit.replace({2:0})

sex_unprev_pred_no_age = data_df[(data_df['sex']==0)]
sex_unprev_pred_no_age = sex_unprev_pred_no_age.y_pred_no_age

sex_unprev_pred_no_sex = data_df[(data_df['sex']==0)]
sex_unprev_pred_no_sex = sex_unprev_pred_no_sex.y_pred_no_sex

# ==============================================================

age_prev_fact = data_df[(data_df['age']==1)]
age_prev_fact = age_prev_fact.credit.replace({2:0})

age_prev_pred_no_age = data_df[(data_df['age']==1)]
age_prev_pred_no_age = age_prev_pred_no_age.y_pred_no_age

age_prev_pred_no_sex = data_df[(data_df['age']==1)]
age_prev_pred_no_sex = age_prev_pred_no_sex.y_pred_no_sex

# --------------------------------------------------------------------------

age_unprev_fact = data_df[(data_df['age']==0)]
age_unprev_fact = age_unprev_fact.credit.replace({2:0})

age_unprev_pred_no_age = data_df[(data_df['age']==0)]
age_unprev_pred_no_age = age_unprev_pred_no_age.y_pred_no_age

age_unprev_pred_no_sex = data_df[(data_df['age']==0)]
age_unprev_pred_no_sex = age_unprev_pred_no_sex.y_pred_no_sex


sex_prev_conf_matrix_no_age = confusion_matrix(sex_prev_fact, sex_prev_pred_no_age, labels=[0, 1])
sex_unprev_conf_matrix_no_age = confusion_matrix(sex_unprev_fact, sex_unprev_pred_no_age, labels=[0, 1])

sex_prev_conf_matrix_no_sex = confusion_matrix(sex_prev_fact, sex_prev_pred_no_sex, labels=[0, 1])
sex_unprev_conf_matrix_no_sex = confusion_matrix(sex_unprev_fact, sex_unprev_pred_no_sex, labels=[0, 1])

age_prev_conf_matrix_no_age = confusion_matrix(age_prev_fact, age_prev_pred_no_age, labels=[0, 1])
age_unprev_conf_matrix_no_age = confusion_matrix(age_unprev_fact, age_unprev_pred_no_age, labels=[0, 1])

age_prev_conf_matrix_no_sex = confusion_matrix(age_prev_fact, age_prev_pred_no_sex, labels=[0, 1])
age_unprev_conf_matrix_no_sex = confusion_matrix(age_unprev_fact, age_unprev_pred_no_sex, labels=[0, 1])

sex_prev_TN_no_age = sex_prev_conf_matrix_no_age[0][0]
sex_prev_FN_no_age = sex_prev_conf_matrix_no_age[1][0]
sex_prev_TP_no_age = sex_prev_conf_matrix_no_age[1][1]
sex_prev_FP_no_age = sex_prev_conf_matrix_no_age[0][1]

sex_unprev_TN_no_age = sex_unprev_conf_matrix_no_age[0][0]
sex_unprev_FN_no_age = sex_unprev_conf_matrix_no_age[1][0]
sex_unprev_TP_no_age = sex_unprev_conf_matrix_no_age[1][1]
sex_unprev_FP_no_age = sex_unprev_conf_matrix_no_age[0][1]
# -----------------------------------------------------------
sex_prev_TN_no_sex = sex_prev_conf_matrix_no_sex[0][0]
sex_prev_FN_no_sex = sex_prev_conf_matrix_no_sex[1][0]
sex_prev_TP_no_sex = sex_prev_conf_matrix_no_sex[1][1]
sex_prev_FP_no_sex = sex_prev_conf_matrix_no_sex[0][1]

sex_unprev_TN_no_sex = sex_unprev_conf_matrix_no_sex[0][0]
sex_unprev_FN_no_sex = sex_unprev_conf_matrix_no_sex[1][0]
sex_unprev_TP_no_sex = sex_unprev_conf_matrix_no_sex[1][1]
sex_unprev_FP_no_sex = sex_unprev_conf_matrix_no_sex[0][1]

# =============================================================

age_prev_TN_no_age = age_prev_conf_matrix_no_age[0][0]
age_prev_FN_no_age = age_prev_conf_matrix_no_age[1][0]
age_prev_TP_no_age = age_prev_conf_matrix_no_age[1][1]
age_prev_FP_no_age = age_prev_conf_matrix_no_age[0][1]

age_unprev_TN_no_age = age_unprev_conf_matrix_no_age[0][0]
age_unprev_FN_no_age = age_unprev_conf_matrix_no_age[1][0]
age_unprev_TP_no_age = age_unprev_conf_matrix_no_age[1][1]
age_unprev_FP_no_age = age_unprev_conf_matrix_no_age[0][1]
# -----------------------------------------------------------
age_prev_TN_no_sex = age_prev_conf_matrix_no_sex[0][0]
age_prev_FN_no_sex = age_prev_conf_matrix_no_sex[1][0]
age_prev_TP_no_sex = age_prev_conf_matrix_no_sex[1][1]
age_prev_FP_no_sex = age_prev_conf_matrix_no_sex[0][1]

age_unprev_TN_no_sex = age_unprev_conf_matrix_no_sex[0][0]
age_unprev_FN_no_sex = age_unprev_conf_matrix_no_sex[1][0]
age_unprev_TP_no_sex = age_unprev_conf_matrix_no_sex[1][1]
age_unprev_FP_no_sex = age_unprev_conf_matrix_no_sex[0][1]

sex_prev_FPR_no_age = sex_prev_FP_no_age / (sex_prev_FP_no_age + sex_prev_TN_no_age)
sex_prev_FNR_no_age = sex_prev_FN_no_age / (sex_prev_FN_no_age + sex_prev_TP_no_age)

sex_unprev_FPR_no_age = sex_unprev_FP_no_age / (sex_unprev_FP_no_age + sex_unprev_TN_no_age)
sex_unprev_FNR_no_age = sex_unprev_FN_no_age / (sex_unprev_FN_no_age + sex_unprev_TP_no_age)

sex_prev_FPR_no_sex = sex_prev_FP_no_sex / (sex_prev_FP_no_sex + sex_prev_TN_no_sex)
sex_prev_FNR_no_sex = sex_prev_FN_no_sex / (sex_prev_FN_no_sex + sex_prev_TP_no_sex)

sex_unprev_FPR_no_sex = sex_unprev_FP_no_sex / (sex_unprev_FP_no_sex + sex_unprev_TN_no_sex)
sex_unprev_FNR_no_sex = sex_unprev_FN_no_sex / (sex_unprev_FN_no_sex + sex_unprev_TP_no_sex)

age_prev_FPR_no_age = age_prev_FP_no_age / (age_prev_FP_no_age + age_prev_TN_no_age)
age_prev_FNR_no_age = age_prev_FN_no_age / (age_prev_FN_no_age + age_prev_TP_no_age)

age_unprev_FPR_no_age = age_unprev_FP_no_age / (age_unprev_FP_no_age + age_unprev_TN_no_age)
age_unprev_FNR_no_age = age_unprev_FN_no_age / (age_unprev_FN_no_age + age_unprev_TP_no_age)

age_prev_FPR_no_sex = age_prev_FP_no_sex / (age_prev_FP_no_sex + age_prev_TN_no_sex)
age_prev_FNR_no_sex = age_prev_FN_no_sex / (age_prev_FN_no_sex + age_prev_TP_no_sex)

age_unprev_FPR_no_sex = age_unprev_FP_no_sex / (age_unprev_FP_no_sex + age_unprev_TN_no_sex)
age_unprev_FNR_no_sex = age_unprev_FN_no_sex / (age_unprev_FN_no_sex + age_unprev_TP_no_sex)

In [55]:
# Report statistical parity and equalized odds using "sex" as the sensitive attribute
sex_statistical_parity_no_age = (sex_pred_no_age_unp / female_All) - (sex_pred_no_age_p / male_All)
sex_statistical_parity_no_sex = (sex_pred_no_sex_unp / female_All) - (sex_pred_no_sex_p / male_All)

sex_equalized_odds_no_age = ((sex_unprev_FPR_no_age-sex_prev_FPR_no_age)+(sex_unprev_FNR_no_age-sex_prev_FNR_no_age)) / 2
sex_equalized_odds_no_sex = ((sex_unprev_FPR-sex_prev_FPR)+(sex_unprev_FNR-sex_prev_FNR)) / 2

# Report statistical parity and equalized odds using "age" as the sensitive attribute
age_statistical_parity_no_age = (age_pred_no_age_unp / young_All) - (age_pred_no_age_p / old_All)
age_statistical_parity_no_sex = (age_pred_no_sex_unp / young_All) - (age_pred_no_sex_p / old_All)

age_equalized_odds_no_age = ((age_unprev_FPR_no_age-age_prev_FPR_no_age)+(age_unprev_FNR_no_age-age_prev_FNR_no_age)) / 2
age_equalized_odds_no_sex = ((age_unprev_FPR_no_sex-age_prev_FPR_no_sex)+(age_unprev_FNR_no_sex-age_prev_FNR_no_sex)) / 2

print('sex_statistical_parity_no_age', round(sex_statistical_parity_no_age, 7))
print('sex_statistical_parity_no_sex', round(sex_statistical_parity_no_sex, 7))
print('age_statistical_parity_no_age', round(age_statistical_parity_no_age, 7))
print('age_statistical_parity_no_sex', round(age_statistical_parity_no_sex, 7))

print("================================")

print('sex_equalized_odds_no_age', round(sex_equalized_odds_no_age, 7))
print('sex_equalized_odds_no_sex', round(sex_equalized_odds_no_sex, 7))
print('age_equalized_odds_no_age', round(age_equalized_odds_no_age, 7))
print('age_equalized_odds_no_sex', round(age_equalized_odds_no_sex, 7))

sex_statistical_parity_no_age -0.1177186
sex_statistical_parity_no_sex -0.1054698
age_statistical_parity_no_age -0.1334553
age_statistical_parity_no_sex -0.1863106
sex_equalized_odds_no_age -0.0645331
sex_equalized_odds_no_sex -0.0728703
age_equalized_odds_no_age -0.0134589
age_equalized_odds_no_sex -0.022873




| 3.4 Results     | Sex | Age   |
| :---        |    :----:   |          ---: |
| **statistical_parity**  | -0.1256194       | -0.1874857   |
| **equalized_odds**   | -0.0728703       | -0.022056      |

**TODO:** Compare the metrics obtained here with those obtained in Step 3.4. What can you say about whether or not using the sensitive attribute have an effect on bias in model predictions for this example?

Both of the two values are closing more than 0, so I expect that sensitive attributes affect bias in model predictions.

### (**Extra Credit**) Step 5: Using AIF360 to compute fairness metrics

In this step, we will use the "aif360" package to detect bias in the dataset (AIF360 is IBM's AI Fairness toolkit) to evaluate fairness of the learned model. This package requires the data to be in a certain format, which will be clear as we walk through the following.

In [56]:
# You should need to run this package installation only once. After the first time, you can comment it out
# !pip install aif360==0.2.2

In [ ]:
!pip install aif360==0.2.2

In [57]:
from aif360.datasets import GermanDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.postprocessing import EqOddsPostprocessing

from aif360.explainers import MetricTextExplainer, MetricJSONExplainer

 Let us use "age" as the sensitive attribute and defined privileged and unprivileged values. We use the GermanDataset in aif360 to do our analysis.

In [58]:
# Store definitions of privileged and unprivileged groups
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset_orig = GermanDataset(protected_attribute_names=['age'],           
                             privileged_classes=[lambda x: x >= 25], 
                             features_to_drop=['personal_status', 'sex'])      # age >=25 is considered privileged


We will convert the dataset to a Pandas dataframe, extract the input features (X) and target variable (y).

In [59]:
dataset_orig_df = dataset_orig.convert_to_dataframe()[0]
x_train = dataset_orig_df.drop("credit", axis=1)
y_train = dataset_orig_df.credit.replace({2:0}) 

First, we will use our model to calculate predicted values for the data and attach them as a new column in the data frame. 

In [60]:
# Copy the dataset
preds_df = dataset_orig_df.copy()

# model = LogisticRegression(C=0.5, penalty="l1", solver='liblinear')
model.fit(x_train, y_train)

# Calculate predicted values
preds_df['credit'] = model.predict(x_train)

# Recode the predictions so that they match the format that the dataset was originally provided in 
# (1 = good credit, 2 = bad credit)
preds_df['credit'] = preds_df.credit.replace({0:2})

Then we will create an object of the aif360 StandardDataset class. You can read more about this in the documentation:
https://aif360.readthedocs.io/en/latest/modules/standard_datasets.html#aif360.datasets.StandardDataset

In [1]:
orig_aif360 = StandardDataset(dataset_orig_df, label_name='credit', protected_attribute_names=['age'], 
                privileged_classes=[[1]], favorable_classes=[1])

preds_aif360 = StandardDataset(preds_df, label_name='credit', protected_attribute_names=['age'], 
                privileged_classes=[[1]], favorable_classes=[1])

NameError: name 'StandardDataset' is not defined

Let us calculate some fairness metrics on the training data using the `ClassificationMetric` function in aif360. 

In [62]:
orig_vs_preds_metrics = ClassificationMetric(orig_aif360, preds_aif360,
                                                   unprivileged_groups=unprivileged_groups,
                                                   privileged_groups=privileged_groups)

In [63]:
# For example, we can compute statistical parity difference as below:
print("Statistical parity difference: ", orig_vs_preds_metrics.statistical_parity_difference())

Statistical parity difference:  -0.20845590264907443


**TODO:** Compare the value obtained above with those obtained in steps 4.2 and 3.4 above. You might observe a difference in the performance due to the way the dataset has been created within aif360.

This value of statistical parity is less than 4.2 and 3.4, which shows a more biased prediction.

Feel free to explore some of the other fairness metrics that we discussed in class. You can find a list of fairness metrics currently supported by aif360 here: https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html.

# Submitting this Assignment Notebook

Once complete, please submit your assignment notebook as an attachment under "Assignments > Assignment 2" on Brightspace. You can download a copy of your notebook using ```File > Download .ipynb```. Please ensure you submit the `.ipynb` file (and not a `.py` file).